<a href="https://colab.research.google.com/github/Sergey-Kiselev-dev/NN_01_Keras/blob/main/NN_01_03b_GradDesc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Сеть для классификации чисел

Подгрузим данные из стандартных датасетов из keras.

Датасет называется MNIST и представляет из себя черно-белые изображения 28 на 28 пикселей.

In [ ]:
import numpy as np
import tensorflow as tf
from keras.datasets import mnist

(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train.shape, X_test.shape

In [ ]:
X_train[0].shape

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 5, figsize=(15, 10))

for i in range(5):
    ax[i].imshow(X_train[i], cmap='gray')
    ax[i].axis('off')

In [ ]:
y_train[:5]

Но давайте упростим себе задачу и возьмем для обучения только два класса, чтобы сделать задачу бинарной классификации.

In [ ]:
idxs = np.where((y_train == 0) | (y_train == 1))
y_train = y_train[idxs]
X_train = X_train[idxs]

X_train.shape, y_train.shape

И тоже самое для теста.

In [ ]:
idxs = np.where((y_test == 0) | (y_test == 1))
y_test = y_test[idxs]

X_test = X_test[idxs]

X_test.shape, y_test.shape

Убедимся, что теперь у нас только 0, либо 1.

In [ ]:
fig, ax = plt.subplots(1, 5, figsize=(15, 10))

for i in range(5):
    ax[i].imshow(X_train[i], cmap='gray')
    ax[i].axis('off')

In [ ]:
y_train[:5]

Нормируем данные, сейчас обойдемся без MinMaxScaler из sklearn, а воспользуемся делением на 255, т.к. сейчас изображения представлены пикселями в диапазоне от 0 до 255, а для нейросети комфортней обучаться на диапазоне от 0 до 1.

In [ ]:
print(X_train.min(), X_train.max())

X_train = X_train / 255.0
X_test = X_test / 255.0

print(X_train.min(), X_train.max())

Так же нужно видоизменить метку класса, сейчас это лейблы 0 или 1, нужно преобразовать в бинарный вид.

Тем самым получаем 2 столбика, где первый - это метка является ли изображение 0 классом, а второй столбик - является ли изображение 1 классом.

In [ ]:
from keras.utils import to_categorical

y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)

y_train[:5]

А чтобы еще легче обучать сетку поменяем масштаб изображений, сейчас они 28 на 28, сделаем меньше, чтобы нейросеть была легче.

In [ ]:
X_train[..., np.newaxis].shape

In [ ]:
import matplotlib.pyplot as plt

X_train_resized = tf.image.resize(X_train[..., np.newaxis], (6, 6))[..., 0]
X_test_resized = tf.image.resize(X_test[..., np.newaxis], (6, 6))[..., 0]

fig, ax = plt.subplots(1, 5, figsize=(15, 10))

for i in range(5):
    ax[i].imshow(X_train_resized[i], cmap='gray')
    ax[i].axis('off')

Для того, что обучить нейронную сеть для любой задачи нужно ответить на три вопроса:
1. Какая архитектура сети?
2. Что оптимизируем?
3. Как обучаем?

### Архитектура сети

Создадим сеть еще сложнее.

Во-первых, на вход поступает изображение 6х6, нужно с ним что-то сделать, так как наша сетку пока не умеет работать с двумерным входом. Здесь нам поможет слой из `keras` `Flatten`, который вытягивает изображение в один вектор, была картинка 6x6, а станет вектором с размерностью 36.

Во-вторых, на выходе не что-то одно, а две вероятности быть или не быть определенным классом.

А значит на выходе имеем два нейрона, каждый из которых отвечает за класс.

input_1 --> 0 --> proba 0

input_2 --> 0 --> proba 1

...

input_36 -->

В-третьих, на выходном слое нужно использовать другую функцию активации, а именно sigmoid, так как она позволяет решать задачу бинарной классификации очень хорошо.

In [ ]:
from keras.layers import Dense
from keras.models import Sequential

tf.random.set_seed(9)

model = Sequential([
    Flatten(input_shape=(6, 6)),
    Dense(2, activation='sigmoid')
])

model.summary()

### Что оптимизируем

У нас задача бинарной классификации, поэтому берем функцию потерь, которая подходит сюда.

Это к примеру, бинарная кросс-энтропия.

In [ ]:
from keras.losses import binary_crossentropy
from keras.optimizers import SGD


optimizer = SGD(learning_rate=0.1)

model.compile(optimizer=optimizer, loss=binary_crossentropy, metrics=['accuracy'])

### Как оптимизируем

Возьмем тот же самый градиентный спуск со стохастикой.

In [ ]:
model.get_weights()

Сделаем предсказания

In [ ]:
preds = model.predict(X_train_resized)
preds

И берем метку класса, где максимальная вероятность.

In [ ]:
preds_cls = preds.argmax(axis=1)
preds_cls

In [ ]:
from sklearn.metrics import accuracy_score

print(f'train acc: {accuracy_score(y_train, preds_cls)*100:.2f}% ({(y_train == preds_cls).sum()} out of {y_train.shape[0]})')

In [ ]:
for i in range(1):
    with tf.GradientTape() as tape:
        pred = model(X_train_resized)

        loss_value = binary_crossentropy(y_train_cat, pred)

        grads = tape.gradient(loss_value, model.trainable_weights)
        print('Grad are', grads)
        print('_' * 40)

    optimizer.apply_gradients(zip(grads, model.trainable_weights))

Сделаем предсказания

In [ ]:
preds = model.predict(X_train_resized)
preds

И берем метку класса, где максимальная вероятность.

In [ ]:
preds_cls = preds.argmax(axis=1)
preds_cls

In [ ]:
from sklearn.metrics import accuracy_score

print(f'train acc: {accuracy_score(y_train, preds_cls)*100:.2f}% ({(y_train == preds_cls).sum()} out of {y_train.shape[0]})')

Еще одну итерацию проведем

In [ ]:
for i in range(1):
    with tf.GradientTape() as tape:
        pred = model(X_train_resized)

        loss_value = binary_crossentropy(y_train_cat, pred)

        grads = tape.gradient(loss_value, model.trainable_weights)

    optimizer.apply_gradients(zip(grads, model.trainable_weights))

Сделаем предсказания

In [ ]:
preds = model.predict(X_train_resized)
preds

И берем метку класса, где максимальная вероятность.

In [ ]:
preds_cls = preds.argmax(axis=1)
preds_cls

In [ ]:
print(f'train acc: {accuracy_score(y_train, preds_cls)*100:.2f}% ({(y_train == preds_cls).sum()} out of {y_train.shape[0]})')

Сравним предсказания для тестовых данных

In [ ]:
preds = model.predict(X_test_resized)
preds

И берем метку класса, где максимальная вероятность.

In [ ]:
preds_cls = preds.argmax(axis=1)
preds_cls

In [ ]:
print(f'test acc: {accuracy_score(y_test, preds_cls)*100:.2f}% ({(y_test == preds_cls).sum()} out of {y_test.shape[0]})')

И благо можем не руками проводить обучение, а пользоваться методом fit в keras

Можем одновременно прогонять модель на обучающих данных, на которых будут меняться веса и на тестовых, чтобы проверять качество работы сети.

In [ ]:
%%time
model.fit(X_train_resized, y_train_cat,
          validation_data=(X_test_resized, y_test_cat),
          epochs=1)

## Summary

Сегодня обсудили:
1. Для чего нужен градиентный спуск
    - GD - это метод оптимизации
    - Нужен для обучения нейронных сетей
2. Что такое градиент
    - Вектор, показывающий направление наискорейшего роста
3. Как написать свой градиентный спуск
    1. Инициализация начальной точки
    2. Цикл по k = 1,2,3,...:
$$ w_{k} = w_{k-1} - \eta\nabla f(w_{k-1}) $$
